### Tutorial 5- Kaggle Advance House Price Prediction Using Pytorch- Tabular Dataset

https://docs.fast.ai/tabular.html
https://www.fast.ai/2018/04/29/categorical-embeddings/
https://www.fast.ai/2018/04/29/categorical-embeddings/
https://yashuseth.blog/2018/07/22/pytorch-neural-network-for-tabular-data-with-categorical-embeddings/


1. Category Embedding


In [2]:
import pandas as pd

In [73]:
df=pd.read_csv('test.csv',usecols=["MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                         "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"])

In [74]:
df.shape

(1459, 9)

In [75]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,YearBuilt,1stFlrSF,2ndFlrSF
0,20,RH,80.0,11622,Pave,Reg,1961,896,0
1,20,RL,81.0,14267,Pave,IR1,1958,1329,0
2,60,RL,74.0,13830,Pave,IR1,1997,928,701
3,60,RL,78.0,9978,Pave,IR1,1998,926,678
4,120,RL,43.0,5005,Pave,IR1,1992,1280,0


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1459 non-null   int64  
 1   MSZoning     1455 non-null   object 
 2   LotFrontage  1232 non-null   float64
 3   LotArea      1459 non-null   int64  
 4   Street       1459 non-null   object 
 5   LotShape     1459 non-null   object 
 6   YearBuilt    1459 non-null   int64  
 7   1stFlrSF     1459 non-null   int64  
 8   2ndFlrSF     1459 non-null   int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 102.7+ KB


In [77]:
for i in df.columns:
    print("Column name {} and unique values are {}".format(i,len(df[i].unique())))

Column name MSSubClass and unique values are 16
Column name MSZoning and unique values are 6
Column name LotFrontage and unique values are 116
Column name LotArea and unique values are 1106
Column name Street and unique values are 2
Column name LotShape and unique values are 4
Column name YearBuilt and unique values are 106
Column name 1stFlrSF and unique values are 789
Column name 2ndFlrSF and unique values are 407


In [9]:
import datetime
datetime.datetime.now().year


2021

In [10]:
df['Total Years']=datetime.datetime.now().year-df['YearBuilt']

In [11]:
df.drop("YearBuilt",axis=1,inplace=True)

In [12]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', '1stFlrSF', '2ndFlrSF', 'Total Years'],
      dtype='object')

In [14]:
cat_features=["MSSubClass", "MSZoning", "Street", "LotShape"]

In [15]:
from sklearn.preprocessing import LabelEncoder
lbl_encoders={}
lbl_encoders["MSSubClass"]=LabelEncoder()
lbl_encoders["MSSubClass"].fit_transform(df["MSSubClass"])

array([0, 0, 5, ..., 0, 9, 5])

In [16]:
lbl_encoders["MSSubClass"]

LabelEncoder()

In [17]:
from sklearn.preprocessing import LabelEncoder
lbl_encoders={}
for feature in cat_features:
    lbl_encoders[feature]=LabelEncoder()
    df[feature]=lbl_encoders[feature].fit_transform(df[feature])

In [18]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,Total Years
0,0,2,80.0,11622,1,3,896,0,60
1,0,3,81.0,14267,1,0,1329,0,63
2,5,3,74.0,13830,1,0,928,701,24
3,5,3,78.0,9978,1,0,926,678,23
4,11,3,43.0,5005,1,0,1280,0,29
...,...,...,...,...,...,...,...,...,...
1454,12,4,21.0,1936,1,3,546,546,51
1455,12,4,21.0,1894,1,3,546,546,51
1456,0,3,160.0,20000,1,3,1224,0,61
1457,9,3,62.0,10441,1,3,970,0,29


In [19]:
import numpy as np

In [20]:
### Stacking and Converting Into Tensors
cat_features=np.stack([df['MSSubClass'],df['MSZoning'],df['Street'],df['LotShape']],1)
cat_features
                       

array([[0, 2, 1, 3],
       [0, 3, 1, 0],
       [5, 3, 1, 0],
       ...,
       [0, 3, 1, 3],
       [9, 3, 1, 3],
       [5, 3, 1, 3]])

In [21]:
### Convert numpy to Tensors
import torch
cat_features=torch.tensor(cat_features,dtype=torch.int64)
cat_features

tensor([[0, 2, 1, 3],
        [0, 3, 1, 0],
        [5, 3, 1, 0],
        ...,
        [0, 3, 1, 3],
        [9, 3, 1, 3],
        [5, 3, 1, 3]])

In [22]:
#### create continuous variable
cont_features=[]
for i in df.columns:
    if i in ["MSSubClass", "MSZoning", "Street", "LotShape","SalePrice"]:
        pass
    else:
        cont_features.append(i)
cont_features

['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'Total Years']

In [23]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,1stFlrSF,2ndFlrSF,Total Years
0,0,2,80.0,11622,1,3,896,0,60
1,0,3,81.0,14267,1,0,1329,0,63
2,5,3,74.0,13830,1,0,928,701,24
3,5,3,78.0,9978,1,0,926,678,23
4,11,3,43.0,5005,1,0,1280,0,29
...,...,...,...,...,...,...,...,...,...
1454,12,4,21.0,1936,1,3,546,546,51
1455,12,4,21.0,1894,1,3,546,546,51
1456,0,3,160.0,20000,1,3,1224,0,61
1457,9,3,62.0,10441,1,3,970,0,29


In [24]:
for i in cont_features:
    print(df[i].values)

[ 80.  81.  74. ... 160.  62.  74.]
[11622 14267 13830 ... 20000 10441  9627]
[ 896 1329  928 ... 1224  970  996]
[   0    0  701 ...    0    0 1004]
[60 63 24 ... 61 29 28]


In [25]:
### Stacking continuous variable to a tensor
cont_values=np.stack([df[i].values for i in cont_features],axis=1)
cont_values=torch.tensor(cont_values,dtype=torch.float)
cont_values

tensor([[   80., 11622.,   896.,     0.,    60.],
        [   81., 14267.,  1329.,     0.,    63.],
        [   74., 13830.,   928.,   701.,    24.],
        ...,
        [  160., 20000.,  1224.,     0.,    61.],
        [   62., 10441.,   970.,     0.,    29.],
        [   74.,  9627.,   996.,  1004.,    28.]])

In [49]:
cont_values.shape

torch.Size([1229, 5])

In [28]:
### Dependent Feature 


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1229 entries, 0 to 1458
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MSSubClass   1229 non-null   int64  
 1   MSZoning     1229 non-null   int64  
 2   LotFrontage  1229 non-null   float64
 3   LotArea      1229 non-null   int64  
 4   Street       1229 non-null   int64  
 5   LotShape     1229 non-null   int64  
 6   1stFlrSF     1229 non-null   int64  
 7   2ndFlrSF     1229 non-null   int64  
 8   Total Years  1229 non-null   int64  
dtypes: float64(1), int64(8)
memory usage: 96.0 KB


In [31]:
cat_features.shape,cont_values.shape

(torch.Size([1229, 4]), torch.Size([1229, 5]))

In [32]:
len(df['MSSubClass'].unique())

15

In [33]:
#### Embedding Size For Categorical columns
cat_dims=[len(df[col].unique()) for col in ["MSSubClass", "MSZoning", "Street", "LotShape"]]

In [34]:
cat_dims

[15, 5, 2, 4]

In [35]:
embedding_dim= [(x, min(50, (x + 1) // 2)) for x in cat_dims]

In [36]:
embedding_dim

[(15, 8), (5, 3), (2, 1), (4, 2)]

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
embed_representation=nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
embed_representation

ModuleList(
  (0): Embedding(15, 8)
  (1): Embedding(5, 3)
  (2): Embedding(2, 1)
  (3): Embedding(4, 2)
)

In [48]:
cat_features.shape

torch.Size([1229, 4])

In [39]:
cat_featuresz=cat_features[:4]
cat_featuresz

tensor([[0, 2, 1, 3],
        [0, 3, 1, 0],
        [5, 3, 1, 0],
        [5, 3, 1, 0]])

In [41]:
pd.set_option('display.max_rows', 500)
embedding_val=[]
for i,e in enumerate(embed_representation):
    embedding_val.append(e(cat_features[:,i]))

In [42]:
embedding_val

[tensor([[ 1.0185,  0.2574, -0.5205,  ..., -1.1158,  0.5322,  1.2248],
         [ 1.0185,  0.2574, -0.5205,  ..., -1.1158,  0.5322,  1.2248],
         [-0.0946, -1.5431,  0.7199,  ...,  1.7629, -0.4667, -0.9584],
         ...,
         [ 1.0185,  0.2574, -0.5205,  ..., -1.1158,  0.5322,  1.2248],
         [ 0.6630, -1.1766, -0.6425,  ..., -2.9346,  0.4635, -1.1256],
         [-0.0946, -1.5431,  0.7199,  ...,  1.7629, -0.4667, -0.9584]],
        grad_fn=<EmbeddingBackward>),
 tensor([[ 1.6396, -1.7661,  0.0148],
         [ 0.0576,  0.4386,  1.4922],
         [ 0.0576,  0.4386,  1.4922],
         ...,
         [ 0.0576,  0.4386,  1.4922],
         [ 0.0576,  0.4386,  1.4922],
         [ 0.0576,  0.4386,  1.4922]], grad_fn=<EmbeddingBackward>),
 tensor([[0.7422],
         [0.7422],
         [0.7422],
         ...,
         [0.7422],
         [0.7422],
         [0.7422]], grad_fn=<EmbeddingBackward>),
 tensor([[-1.9096,  0.5120],
         [-0.6553,  0.1309],
         [-0.6553,  0.1309],
  

In [45]:
z = torch.cat(embedding_val, 1)
z.shape

torch.Size([1229, 14])

In [46]:
##### Create a Feed Forward Neural Network
layerlist = []
import torch
import torch.nn as nn
import torch.nn.functional as F
class FeedForwardNN(nn.Module):

    def __init__(self, embedding_dim, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x

#### Define Loss And Optimizer

In [58]:
cat_features.shape

torch.Size([1229, 4])

In [46]:
df.shape

(1201, 10)

In [53]:
model=torch.load('HousePrice.pt')

In [54]:
model.eval()

FeedForwardNN(
  (embeds): ModuleList(
    (0): Embedding(15, 8)
    (1): Embedding(5, 3)
    (2): Embedding(2, 1)
    (3): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=19, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=100, out_features=70, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=70, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_featur

In [61]:
loss_function=nn.MSELoss()

cont_values.shape,cat_features.shape

(torch.Size([1229, 5]), torch.Size([1229, 4]))

In [63]:
#### Validate the Test Data
y_pred=""
with torch.no_grad():
    y_pred=model(cat_features,cont_values)

In [55]:
data_verify=pd.DataFrame(y_test.tolist(),columns=["Test"])

In [56]:
data_predicted=pd.DataFrame(y_pred.tolist(),columns=["Prediction"])

In [64]:
y_pred

tensor([[117405.3750],
        [157089.5000],
        [192432.3438],
        ...,
        [152587.1875],
        [139870.1562],
        [220625.8125]])

In [67]:
result=pd.read_csv('test.csv',usecols=["Id"])

In [69]:
result["SalePrice"]=y_pred

ValueError: Length of values (1229) does not match length of index (1459)

In [72]:
result.shape

(1459, 1)